# Polymorphism

In [ ]:
from abc import ABC, abstractmethod
from typing import List, Union, Dict
import datetime
import random

# Abstract base class defining the payment interface
class PaymentProcessor(ABC):
    @abstractmethod
    def process_payment(self, amount: float) -> bool:
        """Process a payment and return success status"""
        pass

    @abstractmethod
    def refund(self, amount: float) -> bool:
        """Process a refund and return success status"""
        pass

    @abstractmethod
    def get_transaction_fee(self) -> float:
        """Return the transaction fee percentage"""
        pass

    def generate_transaction_id(self) -> str:
        """Generate a unique transaction ID"""
        return f"TX{random.randint(100000, 999999)}-{datetime.datetime.now().strftime('%Y%m%d')}"

In [3]:
# Concrete class for Credit Card payments
class CreditCardProcessor(PaymentProcessor):
    def __init__(self, card_number: str, expiry: str, cvv: str):
        self.card_number = card_number[-4:].rjust(16, '*')  # Masked card number
        self.expiry = expiry
        self.cvv = cvv
        self.balance = 1000.0  # Sample starting balance
        self.transaction_history: Dict[str, float] = {}

    def process_payment(self, amount: float) -> bool:
        if amount <= 0:
            return False
        if self.balance >= amount:
            self.balance -= amount
            trans_id = self.generate_transaction_id()
            self.transaction_history[trans_id] = amount
            print(f"Credit Card payment of ${amount:.2f} processed (Trans ID: {trans_id})")
            return True
        print("Insufficient funds")
        return False

    def refund(self, amount: float) -> bool:
        if amount <= 0:
            return False
        self.balance += amount
        trans_id = self.generate_transaction_id()
        self.transaction_history[trans_id] = -amount
        print(f"Credit Card refund of ${amount:.2f} processed (Trans ID: {trans_id})")
        return True

    def get_transaction_fee(self) -> float:
        return 2.9  # 2.9% transaction fee

    def __str__(self) -> str:
        return f"Credit Card ending {self.card_number[-4:]} (Balance: ${self.balance:.2f})"

In [4]:
# Concrete class for PayPal payments
class PayPalProcessor(PaymentProcessor):
    def __init__(self, email: str):
        self.email = email
        self.balance = 500.0
        self.transaction_history: Dict[str, float] = {}

    def process_payment(self, amount: float) -> bool:
        if amount <= 0:
            return False
        if self.balance >= amount:
            self.balance -= amount
            trans_id = self.generate_transaction_id()
            self.transaction_history[trans_id] = amount
            print(f"PayPal payment of ${amount:.2f} processed (Trans ID: {trans_id})")
            return True
        print("Insufficient PayPal balance")
        return False

    def refund(self, amount: float) -> bool:
        if amount <= 0:
            return False
        self.balance += amount
        trans_id = self.generate_transaction_id()
        self.transaction_history[trans_id] = -amount
        print(f"PayPal refund of ${amount:.2f} processed (Trans ID: {trans_id})")
        return True

    def get_transaction_fee(self) -> float:
        return 3.4  # 3.4% transaction fee

    def __str__(self) -> str:
        return f"PayPal account {self.email} (Balance: ${self.balance:.2f})"

In [5]:
# Concrete class for Crypto payments
class CryptoProcessor(PaymentProcessor):
    def __init__(self, wallet_address: str, coin_type: str = "BTC"):
        self.wallet_address = wallet_address
        self.coin_type = coin_type
        self.balance = 2.5  # Sample balance in crypto
        self.transaction_history: Dict[str, float] = {}

    def process_payment(self, amount: float) -> bool:
        # Convert USD to crypto (simplified conversion)
        crypto_amount = amount / 50000  # Assuming 1 BTC = $50,000
        if crypto_amount <= 0:
            return False
        if self.balance >= crypto_amount:
            self.balance -= crypto_amount
            trans_id = self.generate_transaction_id()
            self.transaction_history[trans_id] = crypto_amount
            print(f"{self.coin_type} payment of {crypto_amount:.6f} processed (Trans ID: {trans_id})")
            return True
        print("Insufficient crypto balance")
        return False

    def refund(self, amount: float) -> bool:
        crypto_amount = amount / 50000
        if crypto_amount <= 0:
            return False
        self.balance += crypto_amount
        trans_id = self.generate_transaction_id()
        self.transaction_history[trans_id] = -crypto_amount
        print(f"{self.coin_type} refund of {crypto_amount:.6f} processed (Trans ID: {trans_id})")
        return True

    def get_transaction_fee(self) -> float:
        return 1.5  # 1.5% transaction fee

    def __str__(self) -> str:
        return f"{self.coin_type} wallet {self.wallet_address[-6:]} (Balance: {self.balance:.6f})"

In [6]:
# Payment Handler demonstrating polymorphism
class PaymentHandler:
    def __init__(self):
        self.processors: List[PaymentProcessor] = []

    def add_processor(self, processor: PaymentProcessor):
        self.processors.append(processor)

    def process_transaction(self, amount: float):
        print(f"\nProcessing payment of ${amount:.2f}:")
        for processor in self.processors:
            print(f"\nTrying {processor}")
            if processor.process_payment(amount):
                fee = amount * (processor.get_transaction_fee() / 100)
                print(f"Transaction fee: ${fee:.2f}")
                return True
        return False

    def process_refund(self, amount: float):
        print(f"\nProcessing refund of ${amount:.2f}:")
        for processor in self.processors:
            print(f"Refunding to {processor}")
            processor.refund(amount)

In [7]:
# Demonstration
def main():
    # Create payment processors
    credit_card = CreditCardProcessor("1234567890123456", "12/25", "123")
    paypal = PayPalProcessor("user@example.com")
    crypto = CryptoProcessor("1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa")

    # Create payment handler
    handler = PaymentHandler()
    handler.add_processor(credit_card)
    handler.add_processor(paypal)
    handler.add_processor(crypto)

    # Demonstrate polymorphic behavior
    handler.process_transaction(200.0)
    handler.process_transaction(1500.0)  # Should fail all processors due to insufficient funds
    handler.process_refund(50.0)

if __name__ == "__main__":
    main()


Processing payment of $200.00:

Trying Credit Card ending 3456 (Balance: $1000.00)
Credit Card payment of $200.00 processed (Trans ID: TX783142-20250301)
Transaction fee: $5.80

Processing payment of $1500.00:

Trying Credit Card ending 3456 (Balance: $800.00)
Insufficient funds

Trying PayPal account user@example.com (Balance: $500.00)
Insufficient PayPal balance

Trying BTC wallet DivfNa (Balance: 2.500000)
BTC payment of 0.030000 processed (Trans ID: TX908512-20250301)
Transaction fee: $22.50

Processing refund of $50.00:
Refunding to Credit Card ending 3456 (Balance: $800.00)
Credit Card refund of $50.00 processed (Trans ID: TX389866-20250301)
Refunding to PayPal account user@example.com (Balance: $500.00)
PayPal refund of $50.00 processed (Trans ID: TX260922-20250301)
Refunding to BTC wallet DivfNa (Balance: 2.470000)
BTC refund of 0.001000 processed (Trans ID: TX605082-20250301)
